##### The cell below is for you to keep track of the libraries used and install those libraries quickly
##### Ensure that the proper library names are used and the syntax of `%pip install PACKAGE_NAME` is followed

In [2]:
#%pip install pandas 
#%pip install matplotlib
# add commented pip installation lines for packages used as shown above for ease of testing
# the line should be of the format %pip install PACKAGE_NAME 

## **DO NOT CHANGE** the filepath variable
##### Instead, create a folder named 'data' in your current working directory and 
##### have the .parquet file inside that. A relative path *must* be used when loading data into pandas

In [3]:
# Can have as many cells as you want for code
import pandas as pd
import pyarrow.parquet as pq
filepath = "./data/catB_train.parquet" 
# the initialised filepath MUST be a relative path to a folder named data that contains the parquet file

### **ALL** Code for machine learning and dataset analysis should be entered below. 
##### Ensure that your code is clear and readable.
##### Comments and Markdown notes are advised to direct attention to pieces of code you deem useful.

### 1. Data Cleaning 
##### Imputing NA values

In [5]:
df = pd.read_parquet('datathon.parquet')

FileNotFoundError: [Errno 2] No such file or directory: 'datathon.parquet'

In [4]:
#imputing missing values for client information
clnt_info_columns = ['race_desc', 'ctrycode_desc', 'cltsex_fix', 'clntnum', 'clttype', 'stat_flag', 'min_occ_date', 'cltdob_fix']

for clnt_info_column in clnt_info_columns:
    df[clnt_info_column] = df[clnt_info_column].fillna(df[clnt_info_column].mode()[0])

df[['race_desc', 'ctrycode_desc', 'cltsex_fix', 'stat_flag', 'clttype']].isna().sum()

NameError: name 'df' is not defined

In [ ]:
#imputing missing client risk and status indicator
clnt_risk_status_columns = ['flg_substandard', 'flg_is_borderline_standard', 'flg_is_revised_term', 'flg_is_rental_flat', 'flg_has_health_claim', 'flg_has_life_claim', 'flg_gi_claim', 'flg_is_proposal', 'flg_with_preauthorisation', 'flg_is_returned_mail']

for clnt_risk_status_column in clnt_risk_status_columns:
    df[clnt_risk_status_column] = df[clnt_risk_status_column].fillna(df[clnt_risk_status_column].mode()[0])

df[['flg_substandard', 'flg_is_borderline_standard', 'flg_is_revised_term', 'flg_is_rental_flat', 'flg_has_health_claim', 'flg_has_life_claim', 'flg_gi_claim', 'flg_is_proposal', 'flg_with_preauthorisation', 'flg_is_returned_mail']].isna().sum()

flg_substandard               0
flg_is_borderline_standard    0
flg_is_revised_term           0
flg_is_rental_flat            0
flg_has_health_claim          0
flg_has_life_claim            0
flg_gi_claim                  0
flg_is_proposal               0
flg_with_preauthorisation     0
flg_is_returned_mail          0
dtype: int64

In [ ]:
#imputing missing client consent and communication preferences
clnt_consent_columns = ['is_consent_to_mail', 'is_consent_to_email', 'is_consent_to_call', 'is_consent_to_sms', 'is_valid_dm', 'is_valid_email']

for clnt_consent_column in clnt_consent_columns:
    df[clnt_consent_column] = df[clnt_consent_column].fillna(df[clnt_consent_column].mode()[0])

df[['is_consent_to_mail', 'is_consent_to_email', 'is_consent_to_call', 'is_consent_to_sms', 'is_valid_dm', 'is_valid_email']].isna().sum()

is_consent_to_mail     0
is_consent_to_email    0
is_consent_to_call     0
is_consent_to_sms      0
is_valid_dm            0
is_valid_email         0
dtype: int64

In [ ]:
#imputing negative values for n_months_last_bought_products, no missing values
median_without_negatives = df['n_months_last_bought_products'][df['n_months_last_bought_products'] >= 0].median()

df['n_months_last_bought_products'] = df['n_months_last_bought_products'].apply(lambda x: x if x >= 0 else median_without_negatives)

#flg_latest_being_lapse & flg_latest_being_cancel do not have missing values

#imputing recency_lapse with 9999 if flg_latest_being_lapse == 0, otherwise median 
df.loc[df['flg_latest_being_lapse'] == 0, 'recency_lapse'] = 9999
df.loc[(df['flg_latest_being_lapse'] == 1) & (df['recency_lapse'].isna()), 'recency_lapse'] = 0
df['flg_latest_being_lapse'].isna().sum()

#doing the same for cancellations
df.loc[df['flg_latest_being_cancel'] == 0, 'recency_cancel'] = 9999
df.loc[(df['flg_latest_being_cancel'] == 1) & (df['recency_cancel'].isna()), 'recency_cancel'] = 0
df['flg_latest_being_cancel'].isna().sum()

#tot_inforce_pols has no missing values

#imputing missing values of tot_cancel_pols
df.loc[df['flg_latest_being_cancel'] == 0, 'tot_cancel_pols'] = 0
df.loc[(df['tot_cancel_pols'].isna()) & (df['flg_latest_being_cancel'] == 1), 'tot_cancel_pols'] = 1

#imputing missing values of f_ever_declined_la
df['f_ever_declined_la'].fillna(0, inplace=True)

In [1]:
df = pd.read_parquet(filepath)

# Convert the data type of the column to the correct data type
df['hh_size_est'].replace('>4', 5, inplace=True)

income_mapping = {
    'A.ABOVE200K': 200001,  
    'B.100K-200K': 150000,  
    'C.60K-100K': 80000,
    'D.30K-60K': 45000,
    'E.BELOW30K': 15000
}

# Apply the mapping to the 'annual_income' column
df['annual_income_est'].replace(income_mapping, inplace=True)

# Fill in missing values with the median 
col_names_fill_median = ['hh_20', 'pop_20', 'hh_size', 'hh_size_est', 'annual_income_est']

for c in col_names_fill_median:
    median_value = df[c].median()
    df[c].fillna(median_value, inplace=True)

# Fill in missing values with the mode 
col_names_fill_mode = ['is_housewife_retiree', 'is_sg_pr', 'is_class_1_2', 'is_dependent_in_at_least_1_policy']

for c in col_names_fill_mode:
    mode_value = df[c].mode()[0]
    df[c].fillna(mode_value, inplace=True)

# Fill in missing values with 0
col_names_fill_zero = ['flg_affconnect_show_interest_ever', 'flg_affconnect_ready_to_buy_ever', 'flg_affconnect_lapse_ever', 'affcon_visit_days', 'clmcon_visit_days']
for c in col_names_fill_zero:
    df[c].fillna(0, inplace=True)

# Fill in missing values with 9999
col_names_fill_9999 = ['n_months_since_visit_affcon', 'recency_clmcon', 'recency_clmcon_regis']
for c in col_names_fill_9999:
    df[c].fillna(9999, inplace=True)

# Convert target col to 0 or 1 (binary)
df["f_purchase_lh"].fillna(0, inplace=True)

NameError: name 'pd' is not defined

In [49]:
duplicated_clntnum_count = df['clntnum'].duplicated(keep=False).sum()
print(f"Number of duplicated 'clntnum' values: {duplicated_clntnum_count}")

Number of duplicated 'clntnum' values: 0


## The cell below is **NOT** to be removed
##### The function is to be amended so that it accepts the given input (dataframe) and returns the required output (list). 
##### It is recommended to test the function out prior to submission
-------------------------------------------------------------------------------------------------------------------------------
##### The hidden_data parsed into the function below will have the same layout columns wise as the dataset *SENT* to you
##### Thus, ensure that steps taken to modify the initial dataset to fit into the model are also carried out in the function below

In [ ]:
def testing_hidden_data(hidden_data: pd.DataFrame) -> list:
    '''DO NOT REMOVE THIS FUNCTION.

The function accepts a dataframe as input and return an iterable (list)
of binary classes as output.

The function should be coded to test on hidden data
and should include any preprocessing functions needed for your model to perform. 
    
All relevant code MUST be included in this function.'''
    result = [] 
    return result

##### Cell to check testing_hidden_data function

In [ ]:
# This cell should output a list of predictions.
test_df = pd.read_parquet(filepath)
test_df = test_df.drop(columns=["f_purchase_lh"])
print(testing_hidden_data(test_df))

[]


### Please have the filename renamed and ensure that it can be run with the requirements above being met. All the best!